In [127]:
import miner2
import miner2.miner as miner
import os, sys, datetime
import dill
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter

%matplotlib inline

GDC_download_folder = '/Users/user/Projects/melanoma/data/melanoma_data'
results = '/Users/user/Projects/melanoma/Results/'

dill.load_session(results+'subtype_discovery.dill')

#Variables
    #num_cores = 4
    #min_number_genes = 6
    #min_correlation = 0.2
    #
    #expressionData (Dataframe of filtered genes against cases)
    #coexpressionDict (Revised list of clusters formated as a dictionary)
    #mechanistic_output (Dictionary of dictionaries???)
    #referenceDictionary = mechanistic_output
    
pd.set_option('display.max_rows', 500)

In [128]:
#load survival data
survivalSKCM = pd.read_csv("/Users/user/Projects/melanoma/data/clinical.cart.2019-08-14/clinical.tsv", sep='\t', index_col=0,header=0)
survivalDfSKCM = survivalSKCM.iloc[:,7:9]
survivalDfSKCM.columns = ["status", "duration"]
survivalDfSKCM = survivalDfSKCM.loc[(survivalDfSKCM.status == 'Dead') | (survivalDfSKCM.status == 'Alive')]
survivalDfSKCM.loc[survivalDfSKCM.status == 'Alive', 'duration'] = 0
survivalDfSKCM.loc[:, 'duration'] = survivalDfSKCM.loc[:, 'duration'].astype(int)
# overExpressedMembersMatrixMMRF = overExpressedMembersMatrix
# overExpressedMembersMatrixMMRF = overExpressedMembersMatrixMMRF.loc[orderedOverExpressedMembers.index,:]

In [163]:
def kaplanMeier(df, groups=None, statusCol='status', durationCol='duration'):
    allSurvival = df.copy()
#     survivalGroups = np.array(groups)
    
    if groups == None:
        durations = []
        event_observed = []
    
        for status, days in allSurvival.values:
            if status == 'Dead':
                durations.append(int(days/365))
                event_observed.append(1)
            elif status == 'Alive':
                durations.append(max(df.loc[:, 'duration'].values)/365+1)
                event_observed.append(0)
    
        kmf = KaplanMeierFitter()
        kmf.fit(durations, event_observed, label='KM Estimate')

        kmf.plot(ci_show=False)
    
    elif groups is not None:
        for subtype in groups:
            tempDf = allSurvival.loc[subtype]
            print(subtype)
            print(tempDf)
            
            durations = []
            event_observed = []

            for status, days in allSurvival.values:
                if status == 'Dead':
                    durations.append(int(days/365))
                    event_observed.append(1)
                elif status == 'Alive':
                    durations.append(max(df.loc[:, 'duration'].values)/365+1)
                    event_observed.append(0)

            kmf = KaplanMeierFitter()
            kmf = kmf.fit(durations, event_observed, label='KM Estimate')
            
            
            kmf.plot(ci_show=False)
    
    return kmf
#             kmEstimate = kaplanMeier(groupdDf)

In [173]:
tempGroup = np.array(groups[1])
groupDf = survivalDfSKCM.loc[tempGroup]


KeyError: "None of [Index(['9c77be54-d58e-4dec-8894-acb71a12cac5',\n       '73ca07ee-562f-4aef-ab6e-52d7e63b1889',\n       'd1aef548-e9d1-41ff-882d-7e781413e8b4',\n       'f31c68cc-7ef2-4f72-9102-12dd0b73c1e9',\n       'd860679e-7bee-4c69-a9f1-cbf70e643d8f',\n       'cfe24c0a-1d21-49bb-8279-838a4f059e8a',\n       '3b0c5062-fe86-49d7-a47e-4c3378c2a11a',\n       '80b4ed3c-fc03-4d9e-92f8-d98a0e6df284',\n       'b3eaff31-b803-47f9-acca-469053767155',\n       'f8e4b067-7347-4600-a355-50f5e22ca1ee',\n       'c62cc001-e373-466e-9305-32420ac84858',\n       '4daeb7ad-987d-4759-9959-bf9d2410dff5',\n       '0934e651-4cea-4450-b95e-25ee99e276f4',\n       '422d6f92-793c-4784-8895-32a8b56da1fb',\n       '9b7db555-eb53-44ed-aa55-82e79e5b1bd8',\n       '6c23c1f8-53a8-4986-8423-71aef05bf8df',\n       '42221037-1a44-416c-8e3e-fc1805b269de',\n       'ac6098f3-b03b-4fd2-a214-cab070b2ccbd',\n       '0d2ee969-027b-4a7e-be4d-c44aec59c23b',\n       '25ae7e1d-5f00-4eeb-a188-747743ab2e2c',\n       '560da967-730

<lifelines.KaplanMeierFitter: fitted with 459 observations, 238 censored>

In [170]:
np.array(groups[1])

array(['9c77be54-d58e-4dec-8894-acb71a12cac5',
       '73ca07ee-562f-4aef-ab6e-52d7e63b1889',
       'd1aef548-e9d1-41ff-882d-7e781413e8b4',
       'f31c68cc-7ef2-4f72-9102-12dd0b73c1e9',
       'd860679e-7bee-4c69-a9f1-cbf70e643d8f',
       'cfe24c0a-1d21-49bb-8279-838a4f059e8a',
       '3b0c5062-fe86-49d7-a47e-4c3378c2a11a',
       '80b4ed3c-fc03-4d9e-92f8-d98a0e6df284',
       'b3eaff31-b803-47f9-acca-469053767155',
       'f8e4b067-7347-4600-a355-50f5e22ca1ee',
       'c62cc001-e373-466e-9305-32420ac84858',
       '4daeb7ad-987d-4759-9959-bf9d2410dff5',
       '0934e651-4cea-4450-b95e-25ee99e276f4',
       '422d6f92-793c-4784-8895-32a8b56da1fb',
       '9b7db555-eb53-44ed-aa55-82e79e5b1bd8',
       '6c23c1f8-53a8-4986-8423-71aef05bf8df',
       '42221037-1a44-416c-8e3e-fc1805b269de',
       'ac6098f3-b03b-4fd2-a214-cab070b2ccbd',
       '0d2ee969-027b-4a7e-be4d-c44aec59c23b',
       '25ae7e1d-5f00-4eeb-a188-747743ab2e2c',
       '560da967-7304-4c94-8d03-0fa10f4ae7eb',
       '4df76